<a href="https://colab.research.google.com/github/Nishasathish13/TheSchoolofAI-END3.0/blob/main/Session%205%20-%20TorchText%20%26%20Advanced%20Concepts/Assignment/Session_5_Assignment_dataset2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from torchtext.datasets import IMDB

In [2]:
help(IMDB)

Help on function IMDB in module torchtext.datasets.imdb:

IMDB(root='.data', split=('train', 'test'))
    IMDB dataset
    
    Separately returns the train/test split
    
    Number of lines per split:
        train: 25000
    
        test: 25000
    
    
    Number of classes
        2
    
    
    Args:
        root: Directory where the datasets are saved.
            Default: .data
        split: split or splits to be returned. Can be a string or tuple of strings.
            Default: ('train', 'test')



In [20]:
train_iter = IMDB(split='train')
next(train_iter)

('neg',
 'I rented I AM CURIOUS-YELLOW from my video store because of all the controversy that surrounded it when it was first released in 1967. I also heard that at first it was seized by U.S. customs if it ever tried to enter this country, therefore being a fan of films considered "controversial" I really had to see this for myself.<br /><br />The plot is centered around a young Swedish drama student named Lena who wants to learn everything she can about life. In particular she wants to focus her attentions to making some sort of documentary on what the average Swede thought about certain political issues such as the Vietnam War and race issues in the United States. In between asking politicians and ordinary denizens of Stockholm about their opinions on politics, she has sex with her drama teacher, classmates, and married men.<br /><br />What kills me about I AM CURIOUS-YELLOW is that 40 years ago, this was considered pornographic. Really, the sex and nudity scenes are few and far be

In [22]:
# or iterate over a loop

#to display only the first 20 lines
for (ln, (label, line)) in enumerate(train_iter):
  print(ln, label, line)
  if ln == 20:
    break


In [ ]:
# dataloader
from torch.utils.data import DataLoader
train_iter = IMDB(split='train')
dataloader = DataLoader(train_iter, batch_size=16, shuffle=False)

next(iter(dataloader))
#displays 8 labels as batch size = 8

In [28]:
#converting the labels "pos", "neg" to 0 and 1 respectively

labellist = ['pos', 'neg']
label_dict = {}
for idx, item in enumerate(labellist):
  label_dict[item]=idx

label = label_dict['neg']
label

In [ ]:
help(DataLoader)

Torchtext has revamped the very basic components of the torchtext library, including vocab, word vectors, tokenizer. These are the basic data processing building blocks for a raw text strings. 

Here is an example for typical NLP data processing with tokenizer and vocabulary. The first step is to build a vocabulary with the raw training dataset. Here we use built in factory function build_vocab_from_iterator which accepts iterator that yield list or iterator of tokens. Users can also pass any special symbols to be added to the vocabulary.

In [6]:
from torchtext.data.utils import get_tokenizer
from torchtext.vocab import build_vocab_from_iterator

tokenizer = get_tokenizer('basic_english')
train_iter = IMDB(split='train')

#to feed in sentences one by one
def yield_tokens(data_iter):
    for _, text in data_iter:
        yield tokenizer(text)

vocab = build_vocab_from_iterator(yield_tokens(train_iter), specials=["<unk>"])

#sets the default index = 0 for the unknown words, i.e., words not in the vocabulary
vocab.set_default_index(vocab["<unk>"])

In [7]:
def st(*transforms):
  def func(txt_io):
    for transform in transforms:
      txt_io = transform(txt_io)
    return txt_io
  return func

In [ ]:
#theschoolofai is the unknown word in our vocab, hence index set to 0
#vocab(['here', 'is', 'an', 'example', 'theschoolofai'])

Prepare the text processing pipeline with the tokenizer and vocabulary. The text and label pipelines will be used to process the raw data strings from the dataset iterators.

In [27]:
#we need to pass our sentences through the tokenizer first to normalise it before sending it the vocab
text_pipeline = lambda x: vocab(tokenizer(x))
label_pipeline = lambda x: int(x)

In [ ]:
#text_pipeline('here is the an example')

In [ ]:
label_pipeline('1')

0

## Generate data batch and iterator

torch.utils.data.DataLoader is recommended for PyTorch users. It works with a map-style dataset that implements the getitem() and len() protocols, and represents a map from indices/keys to data samples. It also works with an iterable dataset with the shuffle argument of False.

Before sending to the model, collate_fn function works on a batch of samples generated from DataLoader. The input to collate_fn is a batch of data with the batch size in DataLoader, and collate_fn processes them according to the data processing pipelines declared previously. Pay attention here and make sure that collate_fn is declared as a top level def. This ensures that the function is available in each worker.

In [35]:
from torch.utils.data import DataLoader
import torch
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

def collate_fn(batch):
    src_batch, tgt_batch = [], []
    for src_sample, tgt_sample in batch:
        src_batch.append(text_transform[SRC_LANGUAGE](src_sample.rstrip("\n")))
        tgt_batch.append(text_transform[TGT_LANGUAGE](tgt_sample.rstrip("\n")))

    src_batch = pad_sequence(src_batch, padding_value=PAD_IDX)
    tgt_batch = pad_sequence(tgt_batch, padding_value=PAD_IDX)
    return src_batch, tgt_batch

EmbeddingBag is a useful feature to consume sparse ids and produce embeddings. Later we are going to see examples where the text entries in the original batch input will be backed into a list and concatenated as a single tensor for the input of nn.EmbeddingBag.

It is doing two things. The first step is to create an embedding and the second step is to reduce (sum/mean/max, according to the "mode" argument) the embedding output across dimension 0. So this is equivalent to torch.nn.functional.embedding, followed by torch.sum/mean/max. However, the conceptual two step process does not reflect how it's actually implemented. Since embedding_bag does not need to return the intermediate result, it doesn't actually generate a Tensor object for the embedding. It just goes straight to computing the reduction, pulling in the appropriate data from the weight argument according to the indices in the input argument. Avoiding the creation of the embedding Tensor allows for better performance

In [ ]:
weight = torch.randn(3, 4)
weight

In [ ]:
indices = torch.tensor([2, 1])
indices

In [ ]:
embedding_dwork = torch.nn.functional.embedding(indices, weight)
embedding_dwork

In [ ]:
embedding_dwork_mean = embedding_dwork.mean(dim=0, keepdim=True)
embedding_dwork_mean

In [ ]:
embedding_bag = torch.nn.functional.embedding_bag(indices, weight, torch.tensor([0]), mode='mean')
embedding_bag

In this example, the text entries in the original data batch input are packed into a list and concatenated as a single tensor for the input of nn.EmbeddingBag. The offset is a tensor of delimiters to represent the beginning index of the individual sequence in the text tensor. Label is a tensor saving the labels of individual text entries.

In [53]:
def collate_batch(batch):
    label_list, text_list, offsets = [], [], [0]
    for (_label, _text) in batch:
        #print(_label)
        #label_list.append(label_pipeline(_label)) #appending the labels to label_list
        label_list.append(label_dict[_label]) #appending the converted labels to label_list
        processed_text = torch.tensor(text_pipeline(_text), dtype=torch.int64)
        text_list.append(processed_text) 
        offsets.append(processed_text.size(0)) #appending the length of the sentence
    label_list = torch.tensor(label_list, dtype=torch.int64)
    offsets = torch.tensor(offsets[:-1]).cumsum(dim=0)
    text_list = torch.cat(text_list)
    return label_list.to(device), text_list.to(device), offsets.to(device)    

In [54]:
train_iter = IMDB(split='train')
dataloader = DataLoader(train_iter, batch_size=16, shuffle=False, collate_fn=collate_batch)

The model is composed of the nn.EmbeddingBag <https://pytorch.org/docs/stable/nn.html?highlight=embeddingbag#torch.nn.EmbeddingBag>__ layer plus a linear layer for the classification purpose. nn.EmbeddingBag with the default mode of "mean" computes the mean value of a “bag” of embeddings. Although the text entries here have different lengths, nn.EmbeddingBag module requires no padding here since the text lengths are saved in offsets.

Additionally, since nn.EmbeddingBag accumulates the average across the embeddings on the fly, nn.EmbeddingBag can enhance the performance and memory efficiency to process a sequence of tensors.

In [55]:
from torch import nn

class TextClassificationModel(nn.Module):

    def __init__(self, vocab_size, embed_dim, num_class):
        super(TextClassificationModel, self).__init__()
        self.embedding = nn.EmbeddingBag(vocab_size, embed_dim, sparse=True)
        self.fc = nn.Linear(embed_dim, num_class)
        self.init_weights()

    def init_weights(self):
        initrange = 0.5
        self.embedding.weight.data.uniform_(-initrange, initrange)
        self.fc.weight.data.uniform_(-initrange, initrange)
        self.fc.bias.data.zero_()

    def forward(self, text, offsets):
        embedded = self.embedding(text, offsets)
        return self.fc(embedded)

We build a model with the embedding dimension of 64. The vocab size is equal to the length of the vocabulary instance. The number of classes is equal to the number of labels,

In [56]:
train_iter = IMDB(split='train')
num_class = len(set([label for (label, text) in train_iter]))
vocab_size = len(vocab)
emsize = 64
model = TextClassificationModel(vocab_size, emsize, num_class).to(device)

In [72]:
import time

def train(dataloader):
    model.train()
    total_acc, total_count = 0, 0
    log_interval = 500
    start_time = time.time()

    for idx, (label, text, offsets) in enumerate(dataloader):
        #print(label)
        optimizer.zero_grad()
        predited_label = model(text, offsets)
        #loss = criterion(predited_label, label)
        loss = criterion(predited_label, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), 0.1)
        optimizer.step()
        total_acc += (predited_label.argmax(1) == label).sum().item()
        total_count += label.size(0)
        if idx % log_interval == 0 and idx > 0:
            elapsed = time.time() - start_time
            print('| epoch {:3d} | {:5d}/{:5d} batches '
                  '| accuracy {:8.3f}'.format(epoch, idx, len(dataloader),
                                              total_acc/total_count))
            total_acc, total_count = 0, 0
            start_time = time.time()

def evaluate(dataloader):
    model.eval()
    total_acc, total_count = 0, 0

    with torch.no_grad():
        for idx, (label, text, offsets) in enumerate(dataloader):
            predited_label = model(text, offsets)
            loss = criterion(predited_label, label)
            total_acc += (predited_label.argmax(1) == label).sum().item()
            total_count += label.size(0)
    return total_acc/total_count

Split the dataset and run the model
we split the training dataset into train/valid sets with a split ratio of 0.95 (train) and 0.05 (valid). Here we use torch.utils.data.dataset.random_split <https://pytorch.org/docs/stable/data.html?highlight=random_split#torch.utils.data.random_split>__ function in PyTorch core library.

CrossEntropyLoss <https://pytorch.org/docs/stable/nn.html?highlight=crossentropyloss#torch.nn.CrossEntropyLoss>__ criterion combines nn.LogSoftmax() and nn.NLLLoss() in a single class. It is useful when training a classification problem with C classes. SGD <https://pytorch.org/docs/stable/_modules/torch/optim/sgd.html>__ implements stochastic gradient descent method as the optimizer. The initial learning rate is set to 5.0. StepLR <https://pytorch.org/docs/master/_modules/torch/optim/lr_scheduler.html#StepLR>__ is used here to adjust the learning rate through epochs.

In [73]:
from torch.utils.data.dataset import random_split
from torchtext.data.functional import to_map_style_dataset
# Hyperparameters
EPOCHS = 10 # epoch
LR = 5  # learning rate
BATCH_SIZE = 64 # batch size for training
  
criterion = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=LR)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, 1.0, gamma=0.1)
total_accu = None
train_iter, test_iter = IMDB()
train_dataset = to_map_style_dataset(train_iter)
test_dataset = to_map_style_dataset(test_iter)
num_train = int(len(train_dataset) * 0.95)
split_train_, split_valid_ = \
    random_split(train_dataset, [num_train, len(train_dataset) - num_train])

train_dataloader = DataLoader(split_train_, batch_size=BATCH_SIZE,
                              shuffle=False, collate_fn=collate_batch)
valid_dataloader = DataLoader(split_valid_, batch_size=BATCH_SIZE,
                              shuffle=True, collate_fn=collate_batch)
test_dataloader = DataLoader(test_dataset, batch_size=BATCH_SIZE,
                             shuffle=True, collate_fn=collate_batch)

for epoch in range(1, EPOCHS + 1):
    epoch_start_time = time.time()
    train(train_dataloader)
    accu_val = evaluate(valid_dataloader)
    if total_accu is not None and total_accu > accu_val:
      scheduler.step()
    else:
       total_accu = accu_val
    print('-' * 59)
    print('| end of epoch {:3d} | time: {:5.2f}s | '
          'valid accuracy {:8.3f} '.format(epoch,
                                           time.time() - epoch_start_time,
                                           accu_val))
    print('-' * 59)

-----------------------------------------------------------
| end of epoch   1 | time: 10.84s | valid accuracy    0.758 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   2 | time: 10.91s | valid accuracy    0.762 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   3 | time: 10.91s | valid accuracy    0.739 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   4 | time: 10.87s | valid accuracy    0.857 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   5 | time: 10.94s | valid accuracy    0.860 
-----------------------------------------------------------
-----------------------------------------------------------
| end of epoch   6 | time: 10.90s |

In [74]:
print('Checking the results of test dataset.')
accu_test = evaluate(test_dataloader)
print('test accuracy {:8.3f}'.format(accu_test))

Checking the results of test dataset.
test accuracy    0.860


In [ ]:
help(get_tokenizer)

Help on function get_tokenizer in module torchtext.data.utils:

get_tokenizer(tokenizer, language='en')
    Generate tokenizer function for a string sentence.
    
    Args:
        tokenizer: the name of tokenizer function. If None, it returns split()
            function, which splits the string sentence by space.
            If basic_english, it returns _basic_english_normalize() function,
            which normalize the string first and split by space. If a callable
            function, it will return the function. If a tokenizer library
            (e.g. spacy, moses, toktok, revtok, subword), it returns the
            corresponding library.
        language: Default en
    
    Examples:
        >>> import torchtext
        >>> from torchtext.data import get_tokenizer
        >>> tokenizer = get_tokenizer("basic_english")
        >>> tokens = tokenizer("You can now install TorchText using pip!")
        >>> tokens
        >>> ['you', 'can', 'now', 'install', 'torchtext', 'using